<a href="https://colab.research.google.com/github/kelvinlien/RS4Thesis/blob/notebooks/python%5Cnotebook%5Ctext%5CContent_based_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import neccessary libraies
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Software.json.gz

--2021-04-17 10:56:17--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Software.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15033606 (14M) [application/octet-stream]
Saving to: ‘meta_Software.json.gz.5’

meta_Software.json. 100%[===================>]  14.34M  8.65MB/s    in 1.7s    

2021-04-17 10:56:19 (8.65 MB/s) - ‘meta_Software.json.gz.5’ saved [15033606/15033606]



In [10]:
### load the meta data

data = []
with gzip.open('meta_Software.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[7])

26790
{'category': [], 'tech1': '', 'description': ['<i>Living with Arts </i> approach to art appreciation supports students acquisition of the essential skills of the course. Mark Getleins vivid narrative, in concert with McGraw-Hills powerful adaptive learning program LearnSmart and CONNECT Art, helps students <b>understand </b>, <b>analyze </b>, and <b>appreciate </b> the ways art work communicates to us in our visual world. In doing so, <i>Living with Art </i> provides the foundation for a life-long appreciation of art, and critical thinking skills that will benefit students far beyond the classroom into their chosen careers. Students are invited to actively join in the discussion from Chapter 1. The narrative engages them in the stories of art, while the integrated digital assessments challenge them to build a fluency in the core vocabulary of the course, practice applying it in guided writing assignments, and ultimately prepare for active class participation and success on critic

In [11]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

26790


In [12]:
#Text Preprocessing
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [13]:
# Apply preprocess functions
target = "description"
df[target] = df[target].astype("string")
df[target] = df[target].apply(_removeNonAscii)
df[target] = df[target].apply(func = make_lower_case)
df[target] = df[target].apply(func = remove_stop_words)
df[target] = df[target].apply(func=remove_punctuation)
df[target] = df[target].apply(func=remove_html)
df

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,[],,,,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,[],[],,HOLT. RINEHART AND WINSTON,[],"25,550 in Software (",[],Software,,</div>,.a-box-inner{background-color:#fff}#alohaBuyBo...,0030672120,NaN
1,[],,b latin rhythms get kids singing spanish b i s...,,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",[],[https://images-na.ssl-images-amazon.com/image...,,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,NaN
2,[],,b connect integrated learning system empowers ...,,Connect with LearnSmart Access Card for Microb...,[],[https://images-na.ssl-images-amazon.com/image...,,McGraw-Hill Science/Engineering/Math,[],"16,900 in Software (",[],Software,,</div>,,007329506X,NaN
3,[],,,,LearnSmart Standalone Access Card for Prescott...,[],[https://images-na.ssl-images-amazon.com/image...,,McGraw-Hill Education,[],"12,986 in Software (",[],Software,,</div>,,0073513458,NaN
4,[],,i anatomy amp physiology revealed cat i ultima...,,Anatomy &amp; Physiology Revealed Student Acce...,"[0323394612, 0323227937, 1118527488]",[https://images-na.ssl-images-amazon.com/image...,,McGraw-Hill Education,[],"14,861 in Software (",[],Software,,</div>,$4.83,0073525758,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26785,"[Software, Children's]",,b features b br 1 beautiful fabulously detaile...,,925 Sterling Silver Angel Wings Heart Pendant ...,"[B07B4B12JX, B008UX1WJ2]",[https://images-na.ssl-images-amazon.com/image...,,17maimeng,[Material: Guaranteed 100% Real Silver+Platinu...,"1,389,844 in Clothing, Shoes & Jewelry (","[B008UX1WJ2, B0094JVCKU, B077J3DR69, B07J4Z659...","<img src=""https://images-na.ssl-images-amazon....",,17maimeng,$24.00,B01HEFZJC2,{}
26786,"[Software, Children's, Material: Guaranteed 10...",,b features b br 1 beautiful fabulously detaile...,,925 Sterling Silver Love Heart Opal Pendant Ne...,[],[https://images-na.ssl-images-amazon.com/image...,,17maimeng,[Material: Guaranteed 100% Real Silver+Platinu...,"1,469,307 in Clothing, Shoes & Jewelry (","[B074M486S8, B01D4H965K, B077L7GGF4, B019D8X0W...","<img src=""https://images-na.ssl-images-amazon....",,17maimeng,$23.20,B01HEFZKEE,{}
26787,"[Software, Digital Software, Antivirus & Secur...",,div mac internet security x9 contains two best...,,Intego Mac Internet Security X9 - 1 Mac - 1 ye...,[],[],,Intego,[Award-winning antivirus software to protect a...,"2,733 in Software (","[B01MF5MTWP, B015724B8M, B07CYFFH4H, B07CY54KL...",Software,,</div>,$39.99,B01HF3G4BS,"{'Downloading:': 'Currently, this item is avai..."
26788,[],,versacheck x9 quickbooks 2016 dna secure 100 c...,,VersaCheck X9 Small and Medium Business 2016 3...,[],[],,Diversified Productivity Solutions Ltd,[],"15,575 in Software (",[],Software,,</div>,$24.39,B01HF41TKI,"{'Shipping Weight:': '1.3 pounds', 'ASIN:': 'B..."


In [14]:
#Building TFIDF model and calculate TFIDF score
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df[target])

In [15]:
# Calculating the similarity measures based on Cosine Similarity
sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
sg

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 2.71302237e-03, ...,
        2.41469958e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.71302237e-03, 1.00000000e+00, ...,
        6.18183294e-04, 0.00000000e+00, 5.39984030e-03],
       ...,
       [0.00000000e+00, 2.41469958e-03, 6.18183294e-04, ...,
        1.00000000e+00, 3.56642896e-02, 9.99592916e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.56642896e-02, 1.00000000e+00, 6.20719116e-03],
       [0.00000000e+00, 0.00000000e+00, 5.39984030e-03, ...,
        9.99592916e-03, 6.20719116e-03, 1.00000000e+00]])

In [16]:
results = {}
for idx, row in df.iterrows():
    similar_indices = sg[idx].argsort()[:-100:-1]
    similar_items = [(sg[idx][i], df['title'][i]) for i in similar_indices]

    # First item is the item itself, so remove it.
    # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, item_id)
    results[row['title']] = similar_items[1:]

In [17]:
def item(title):
    return df.loc[df['title'] == title]["title"].tolist()[0].split(' - ')[0]

In [18]:
# Top 5 recommendation
def recommend(title, num):
    print("Recommending " + str(num) + " products similar to " + item(title) + "...")
    print("-------")
    recs = results[title][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

In [19]:
recommend(title="Connect Art with McGraw-Hill LearnSmart Access Card for Living with Art", num=5)

Recommending 5 products similar to Connect Art with McGraw-Hill LearnSmart Access Card for Living with Art...
-------
Recommended: Connect Art with McGraw-Hill LearnSmart Access Card for Living with Art (score:1.0000000000000002)
Recommended: JumpStart Artist (score:0.21999633446673098)
Recommended: JumpStart Artist (score:0.21999633446673098)
Recommended: Art (DVD) (PC &amp; Mac) [Old Version] (score:0.20643395795384933)
Recommended: SUMMITSOFT Clip Art Editor ( Windows ) (score:0.1920555003219637)
